# The begin

In [1]:
import cv2
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data

import scipy.stats as ss

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm

from PIL import Image, ImageDraw, ImageFont
import PIL

import os

import copy
import random
import time

import tracemalloc

%matplotlib inline

In [ ]:
bert_path = "cointegrated/rubert-tiny2"
vit_path = "google/vit-base-patch16-224-in21k"

In [ ]:
import gc


def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
!pip install sentencepiece
!pip install levenshtein
!pip install transformers

In [ ]:
from transformers import (
    AutoModel,
    AutoTokenizer,
    BertConfig,
    DeiTConfig,
    DeiTModel,
    DeiTFeatureExtractor,
    TrOCRConfig,
    TrOCRProcessor,
    TrOCRForCausalLM,
    ViTConfig,
    ViTModel,
    VisionEncoderDecoderModel,
    VisionEncoderDecoderConfig,
    ViTFeatureExtractor,
)

In [ ]:
from google.colab import drive
drive.mount('./data')

In [ ]:
cd data/MyDrive/ColabNotebooks/Doc_recognition/Passports

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
font_size = 15
# passport_font = "fonts/arial_0.ttf"
passport_font = "fonts/ocrb.ttf"

symbols = " .-1234567890АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
# symbols = " .-1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"

length = 1 + ss.poisson(1).rvs(size=1)
text = ''.join(np.random.choice(list(symbols), size=length))

image = PIL.Image.open("photo_data/backgrounds/white.jpg").resize((224, 32))
w, h = image.size

draw = ImageDraw.Draw(image)
a, b = draw.textsize(text, font=ImageFont.truetype(passport_font, font_size))
print(a, b, w, h)

draw.text((0, 0), text, font=ImageFont.truetype(passport_font, font_size), fill=(50, 50, 50))
image = image.crop((0, 0, a + 1, b + 1)).resize((224, 32))

print(text)
plt.imshow(image)

## Датасет и даталоадер
Первая часть модели отвечает за нахождение региончиков с текстом.  
Происходит патчевая классификация. То есть таргет для картинки - это 2d тензор размера *\[H/P, W/P\]*, где для каждого патча выставлен класс, к которому от относится.  
Классов всего 16: 14 видов текстовых полей + *'photo'* + *'blank'*

In [ ]:
class PassportDataset(Dataset):
    def __init__(self, size, length, processor, w=224, h=32):
        """
        pd_table - path to the dataset
        img_dir - directory with passport images
        w, h - the desirable size of the image (to resize)
        p - patch size
        from_, to_ - set them to take a part of a dataset, not the whole
        """
        self.w, self.h = w, h

        self.processor = processor
        self.size = size
        self.length = length
        self.img = PIL.Image.open("photo_data/backgrounds/cloth.jpg").resize((w, h))
        # self.passport_font = "fonts/arial_0.ttf"
        self.passport_font = "fonts/ocrb.ttf"

        self.transforms = transforms.Compose([
                transforms.Resize((h, w)),
                transforms.ColorJitter(brightness=(0.9, 1.1), contrast=(0.8, 1.25)),
                transforms.ToTensor()
            ])
        self.shape = (w, h)  # shape: x, y - for PIL image

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        font_size = np.random.randint(10, 16)
        symbols = " .-1234567890АБВГДЕЁЖЗИЙКЛМНОПРСТФХЦЧШЩЪЫЬЭЮЯ"
        # symbols = " .-1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        length = self.length + ss.poisson(1).rvs(size=1)
        text = ''.join(np.random.choice(list(symbols), size=length))

        image = self.img.copy()
        w, h = image.size

        draw = ImageDraw.Draw(image)
        a, b = draw.textsize(text, font=ImageFont.truetype(self.passport_font, font_size))
        
        draw.text((0, 0), text, font=ImageFont.truetype(self.passport_font, font_size), fill=(50, 50, 50))
        image = image.crop((0, 0, a + 1, b + 1))

        # tensor = self.transforms(image)
        # del image

        text = torch.tensor(self.processor.tokenizer(text, padding="max_length", max_length=32).input_ids)

        # return self.processor(tensor, return_tensors="pt").pixel_values[0], text
        return self.processor(images=image, return_tensors="pt").pixel_values[0], text

In [ ]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-handwritten')

In [ ]:
train_data = PassportDataset(100, 2, processor)
test_data = PassportDataset(25, 2, processor)

In [ ]:
x, t = train_data[0]
print(t.shape)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=2, shuffle=True)

In [ ]:
for x, t in train_dataloader:
    print(t.shape)
    break

## Обучение itself

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Подготовка к распознаванию

# Свои train и eval

In [ ]:
import Levenshtein as pylev

In [ ]:
def train_full(model, train_data, optimizer, device):
    model.train()
    epoch_loss = 0
    normal_crop_loss = 0.04

    for X, texts in tqdm(train_data, desc="Training"):
        model.zero_grad()
        loss = model(pixel_values=X.to(device), labels=texts.to(device)).loss

        cleanup()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        del X, texts
        cleanup()

    return epoch_loss / len(train_data)


def evaluate_full(model, val_data, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for X, texts in tqdm(val_data, desc="Evaluating"):
            loss = model(pixel_values=X.to(device), labels=texts.to(device)).loss

            cleanup()

            epoch_loss += loss.item()
            del X, texts
            cleanup()

    return epoch_loss / len(val_data)


def count_levenshtein(model, processor, val_data, device):
    model.eval()
    epoch_lev_dist = []
    epoch_cer = []
    with torch.no_grad():
        for X, texts in tqdm(val_data, desc="Evaluating levenshtein"):        
            outputs = model.generate(X.to(device), max_length=32)
            cleanup()

            targets = processor.batch_decode(texts, skip_special_tokens=True)
            preds = processor.batch_decode(outputs, skip_special_tokens=True)

            for pred, target in zip(preds, targets):
                if np.random.randint(100) < 4:
                    print(f"[{pred}] vs [{target}]")
                epoch_lev_dist.append(pylev.distance(pred, target))
                if pred or target:
                    epoch_cer.append(epoch_lev_dist[-1] / max(len(pred), len(target)))

            del X, texts
            cleanup()

    return np.mean(epoch_lev_dist), np.mean(epoch_cer)

# Final train

In [19]:
bert_path = "cointegrated/rubert-tiny2"
vit_path = "google/vit-base-patch16-224-in21k"

In [20]:
tokenizer = AutoTokenizer.from_pretrained(bert_path)
feature_extractor = DeiTFeatureExtractor.from_pretrained(vit_path)

processor = TrOCRProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.image_processor.size = {'height': 32, 'width': 224}

/usr/local/lib/python3.10/dist-packages/transformers/models/deit/feature_extraction_deit.py:28: FutureWarning: The class DeiTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DeiTImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/trocr/processing_trocr.py:44: FutureWarning: The `feature_extractor` argument is deprecated and will be removed in v5, use `image_processor` instead.
  warnings.warn(


In [21]:
processor

TrOCRProcessor:
- image_processor: DeiTFeatureExtractor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "DeiTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 32,
    "width": 224
  }
}

- tokenizer: BertTokenizerFast(name_or_path='cointegrated/rubert-tiny2', vocab_size=83828, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [22]:
config_encoder = ViTConfig.from_pretrained(vit_path)
config_decoder = BertConfig.from_pretrained(bert_path)
config = VisionEncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

recognize_model = VisionEncoderDecoderModel(config)
# set special tokens used for creating the decoder_input_ids from the labels
recognize_model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
recognize_model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
recognize_model.config.vocab_size = recognize_model.config.decoder.vocab_size

# set beam search parameters
recognize_model.config.eos_token_id = processor.tokenizer.sep_token_id
recognize_model.config.max_length = 64
recognize_model.config.early_stopping = True
recognize_model.config.no_repeat_ngram_size = 3
recognize_model.config.length_penalty = 2.0
recognize_model.config.num_beams = 3

In [23]:
# recognize_model

# What if

In [24]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-handwritten')
# processor.image_processor.size = {'height': 32, 'width': 224}

recognize_model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-handwritten')

recognize_model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
recognize_model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
recognize_model.config.vocab_size = recognize_model.config.decoder.vocab_size

# set beam search parameters
recognize_model.config.eos_token_id = processor.tokenizer.sep_token_id
recognize_model.config.max_length = 64
recognize_model.config.early_stopping = True
recognize_model.config.no_repeat_ngram_size = 3
recognize_model.config.length_penalty = 1.0
recognize_model.config.num_beams = 3

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
processor

TrOCRProcessor:
- image_processor: DeiTImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": false,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "DeiTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 384,
    "width": 384
  }
}

- tokenizer: XLMRobertaTokenizerFast(name_or_path='microsoft/trocr-small-handwritten', vocab_size=64002, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_classes = 16
iteration = 0

if iteration > 0:
    if device == 'cuda':
        recognize_model.load_state_dict(torch.load(f'models_backups/rand_just_recognize_latest_{iteration - 1}.pt'))
    else:
        recognize_model.load_state_dict(torch.load(f'models_backups/rand_just_recognize_latest_{iteration - 1}.pt', map_location=torch.device('cpu')))

# crop_model.to(device)
# recognize_model.to(device)
recognize_model.to(device)

optimizer = optim.Adam(recognize_model.parameters(), lr=1e-5)

warmup_scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=1e-3, total_iters=5)
gamma_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.25)
# cosine_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 15)
# main_scheduler = optim.lr_scheduler.ChainedScheduler([cosine_scheduler, gamma_scheduler])
# lr_scheduler = optim.lr_scheduler.SequentialLR(optimizer, schedulers=[warmup_scheduler, main_scheduler], milestones=[10])
lr_scheduler = optim.lr_scheduler.SequentialLR(optimizer, schedulers=[warmup_scheduler, gamma_scheduler], milestones=[10])

lengths = [4, 4, 5, 7, 10]

f"Working on {device}"

'Working on cuda'

In [27]:
X, texts = None, None
for data in train_dataloader:
    X, texts = data

outputs = recognize_model.generate(X.to(device), max_length=5)

targets = processor.batch_decode(texts, skip_special_tokens=True)
preds = processor.batch_decode(outputs, skip_special_tokens=True)

print(targets)
print(preds)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


['6', 'Ж73']
[".'' '", '.']


In [ ]:
for i, length in enumerate(lengths):
    train_data = PassportDataset(500, length, processor)
    test_data = PassportDataset(100, length, processor)
    train_dataloader = DataLoader(train_data, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_data, batch_size=16, shuffle=False)

    EPOCHS = 20
    train_crop_loss_storage, valid_crop_loss_storage = [], []
    train_recognize_loss_storage, valid_recognize_loss_storage = [], []

    valid_recognize_loss = evaluate_full(recognize_model, test_dataloader, device)
    valid_lev_dist, valid_cer = count_levenshtein(recognize_model, processor, test_dataloader, device)
    print('Before training')
    print(f'\t Val. recognizing loss: {valid_recognize_loss:.3f}')
    print(f'\t Val. Levenshein distance: {valid_lev_dist:.3f}')
    print(f'\t Val. Cer: {valid_cer:.3f}')

    best_valid_loss = valid_recognize_loss

    for epoch in range(EPOCHS):
        print(f"Epoch #{epoch + 1}")
        start_time = time.monotonic()

        train_recognize_loss = train_full(recognize_model, train_dataloader, optimizer, device)
        train_recognize_loss_storage.append(train_recognize_loss)

        valid_recognize_loss = evaluate_full(recognize_model, test_dataloader, device)
        valid_recognize_loss_storage.append(valid_recognize_loss)

        if valid_recognize_loss < best_valid_loss:
            best_valid_loss = valid_recognize_loss
            torch.save(recognize_model.state_dict(), f'models_backups/rand_just_recognize_best_{iteration}.pt')
        torch.save(recognize_model.state_dict(), f'models_backups/rand_just_recognize_latest_{iteration}.pt')

        valid_vel_dist, valid_cer = count_levenshtein(recognize_model, processor, test_dataloader, device)

        lr_scheduler.step()
        end_time = time.monotonic()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        print(f'Epoch: {i * 20 + epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\t Val. recognize loss: {valid_recognize_loss:.3f}')
        print(f'\t Val. Levenshein distance: {valid_lev_dist:.3f}')
        print(f'\t Val. Cer: {valid_cer:.3f}')

        cleanup()

        lr_scheduler.step()

Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

Evaluating levenshtein:   0%|          | 0/7 [00:00<?, ?it/s]

[. 'fully.'you'. you'll know you'!. you you you know you.] vs [ЗМ3Ц]
[in] vs [ЭЦШ0]
[. " you-] vs [ВЧХ-]
[not for the work for the workshops that write written that written that the work with the work written that workshops that written workshops that that write workshops that the workshops] vs [5ОГЯ64]
Before training
	 Val. recognizing loss: 20.922
	 Val. Levenshein distance: 23.400
	 Val. Cer: 0.956
Epoch #1


Training:   0%|          | 0/32 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

Evaluating levenshtein:   0%|          | 0/7 [00:00<?, ?it/s]

[Ч] vs [Г.]
[ФЯ] vs [ГХШ Р]
[АА] vs [ПАГТА]
[Ф] vs [ЭГЭ49В]
Epoch: 28 | Epoch Time: 1m 6s
	 Val. recognize loss: 0.664
	 Val. Levenshein distance: 4.140
	 Val. Cer: 0.950
Epoch #9


Training:   0%|          | 0/32 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

# THE END

In [ ]:
def reqognize(crop):
    pixel_values = processor(crop, return_tensors="pt").pixel_values.to(recognize_model.device)
    generated_ids = recognize_model.generate(pixel_values)
    return processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    #cer = cer_metric.compute(predictions=pred_str, references=label_str)
    cer = pylev.distance(pred_str, label_str)

    return {"cer": cer}

In [ ]:
batch_size = 32  # сколько примеров показывем модели за один шаг
report_steps = 200  # раз в сколько шагов печатаем результат
epochs = 200  # сколько раз мы покажем данные модели
early_stop_patience = -1
lr = 5e-05 / 10
lr_decay = 0.95  # 0.8
device = 'cuda'

In [ ]:
pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="./",
    logging_steps=2,
    save_steps=10000,
    eval_steps=200,
    learning_rate=lr,
    num_train_epochs=epochs,
    max_steps=-1,
)

NameError: ignored

In [ ]:
cleanup()
set_random_seed()

recognize_model.to(device)

In [ ]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=recognize_model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=default_data_collator,
)